In [1]:
print("OK!")

OK!


In [2]:
pip install langchain


Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install -U langchain-community

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install pinecone-client


Note: you may need to restart the kernel to use updated packages.


In [5]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers


In [6]:
PINECONE_API_KEY = "f63be812-4281-4286-9639-b709cab0c1d6"
PINECONE_API_ENV = "Serverless"

In [7]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [8]:
pip install pypdf

Note: you may need to restart the kernel to use updated packages.


In [9]:
extracted_data = load_pdf("/Users/him/Desktop/End-to-end-Medical-Chatbot-using-Llama2/data/")

In [10]:
# extracted_data


In [11]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [12]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 7020


In [13]:
# text_chunks

In [14]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [15]:
embeddings = download_hugging_face_embeddings()

/Users/him/anaconda3/envs/car_prediction/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/Users/him/anaconda3/envs/car_prediction/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [16]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [17]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [18]:
# query_result

In [19]:
pinecone_client = pinecone.Pinecone(
    api_key=PINECONE_API_KEY ,
    environment=PINECONE_API_ENV 
)
# Define the index name
index_name = "mchatbot"


In [20]:
# Assuming `text_chunks` is a list of text content
texts = [t.page_content for t in text_chunks]



In [21]:
pinecone.api_key= PINECONE_API_KEY
pinecone.environment = PINECONE_API_ENV

In [22]:

import pinecone
from transformers import AutoTokenizer, AutoModel
import torch

In [23]:
PINECONE_API_KEY = "f63be812-4281-4286-9639-b709cab0c1d6"
PINECONE_API_ENV = "Serverless"

In [24]:
pinecone_client = pinecone.Pinecone(api_key=PINECONE_API_KEY, environment=PINECONE_API_ENV)

In [25]:

index_name="mchatbot"

In [31]:
import os
import pinecone

# Set the environment variables for Pinecone API key and environment
os.environ['PINECONE_API_KEY'] = 'f63be812-4281-4286-9639-b709cab0c1d6'
os.environ['PINECONE_ENV'] = 'Serverless'

In [34]:
#Creating Embeddings for Each of The Text Chunks & storing
docsearch=Pinecone.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

In [35]:
#If we already have an index we can load it like this
docsearch=Pinecone.from_existing_index(index_name, embeddings)

query = "What are Allergies"

docs=docsearch.similarity_search(query, k=3)

print("Result", docs)

Result [Document(page_content="GALE ENCYCLOPEDIA OF MEDICINE 2 117Allergies\nAllergic rhinitis is commonly triggered by\nexposure to household dust, animal fur,or pollen. The foreign substance thattriggers an allergic reaction is calledan allergen.\nThe presence of an allergen causes the\nbody's lymphocytes to begin producingIgE antibodies. The lymphocytes of an allergy sufferer produce an unusuallylarge amount of IgE.\nIgE molecules attach to mast\ncells, which contain histamine.HistaminePollen grains\nLymphocyte\nFIRST EXPOSURE"), Document(page_content='allergens are the following:\n• plant pollens\n• animal fur and dander\n• body parts from house mites (microscopic creatures\nfound in all houses)\n• house dust• mold spores• cigarette smoke• solvents• cleaners\nCommon food allergens include the following:\n• nuts, especially peanuts, walnuts, and brazil nuts\n• fish, mollusks, and shellfish• eggs• wheat• milk• food additives and preservatives\nThe following types of drugs commonly ca

In [36]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [37]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [38]:
pip install ctransformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [39]:
llm=CTransformers(model="/Users/him/Desktop/End-to-end-Medical-Chatbot-using-Llama2/model",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [ ]:
import threading

# Function to run the qa query
def run_qa_query(query, result_container):
    result_container.append(qa({"query": query}))

while True:
    user_input = input("Input Prompt: ")
    
    if user_input.lower() == "exit":
        print("Exiting the loop.")
        break

    result_container = []
    query_thread = threading.Thread(target=run_qa_query, args=(user_input, result_container))
    query_thread.start()

    query_thread.join(timeout=10)  # Timeout after 10 seconds

    if query_thread.is_alive():
        print("The query took too long to respond and was terminated.")
        query_thread.join()  # Ensure the thread has finished
    else:
        if result_container:
            result = result_container[0]
            print("Response: ", result["result"])
        else:
            print("No response received.")
